In [12]:
from pyspark.sql.functions import substring_index
from pyspark.sql import SparkSession, Row
import re
import pandas as pd
#Per iniziare, definisco i dataframe dei libri e dei tipi dei libri.
#DOMANDA: cosa si intende per "Manifestation"?

file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
DFitems = spark.read.csv(file,header=True)
DFitems.show()

file = "/data/SMARTDATA/books/BCT_prestiti/item_media_types.csv"
DFtypes = spark.read.csv(file,header=True)
DFtypes.show()

#Come posso constatare, è possibile joinare media types e libri tramite l'id media types per vedere a quale genere
#si riferisce il libro e filtrare così i generi non necessari.

+-------+----------------+-------------+---------------+--------------+
|item_id|manifestation_id|item_media_id|home_library_id|inventory_date|
+-------+----------------+-------------+---------------+--------------+
| 288788|          324685|            Q|             27|    2007-06-04|
|1987167|           18421|            S|             13|    2013-01-28|
|2555210|          747441|            F|              2|    2016-09-15|
| 218132|            3490|            F|              2|    2008-03-12|
|1009634|          233431|            F|             19|    2004-05-27|
| 121499|          286272|            F|             27|    2010-09-01|
|2400369|          270193|            F|             16|    2016-01-27|
| 103294|          365773|            F|              2|    2010-06-08|
| 164444|          269506|            F|             20|    2006-05-23|
|2269659|          625603|            A|              3|    2015-02-20|
|1488598|          449444|            T|              2|    2011

In [15]:
#FILTRAGGIO NUMERO 1: GENERI NON NECESSARI
#Per un lettore "medio" credo sia ragionevole filtrare via i materiali multimediali (DVD, VHS, audiolibri, proiezioni
#e altri materiali elettronici), i libri in braille (che rappresentano, immagino, una forte minoranza all'interno del catalogo)
#e i materiali cartografici poiché immagino siano più usati da esperti che da lettori interessati alle raccomandazioni. Discorso
#analogo per le lettere autografe.
#Le microforme (riassunti di libri in versione compatta) credo siano più interessanti per gli studenti che, in quanto tali,
#non dovrebbero aver bisogno di raccomandazioni.
#I periodici (e tutti i magazine) non devono essere contati dal nostro sistema come libri e vanno dunque filtrati (filtraggio che
#dovrebbe anche consentirci di avere una top k più pulita).
#Per cui, post filtraggio, i generi che rimangono sono le monografie, i manoscritti e (forse) "grafica" e "oggetto". Devo
#controllare cosa si intenda con queste due categorie.
#NOTA: Come risalire ai titoli dei libri da items.csv? NOTA2: per qualche motivo, item_id e manifestation_id sono differenti

file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
DFmanifestations = spark.read.csv(file,header=True)
DFmanifestations.show(50, False)


+----------------+----------------+------------+-------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+----------------------------------------+-------------------+-------------------+
|manifestation_id|edition_language|edition_date|ISBNISSN     |EAN          |title                                                                                                                                                                                                                                                                  |author                                         |publisher                               |loanable_since     |date_created       |
+----------------+----------------+------------+------------

In [2]:
#Filtraggio numero 2: amalgamare i libri uguali ma di edizioni diverse sotto un unico ID. Ciò sarà utile
#per evitare di considerare libri praticamente uguali come diversi.
#COME SI PROCEDE? Potrei joinare loans a filteredmanifestations usando item_id e poi applicare una trasformazione
#in cui, a parità di titolo, l'id venga cambiato in modo da risultare uguale.
#Cosa abbiamo in loans.csv?

file = "/data/SMARTDATA/books/BCT_prestiti/loans.csv"
DFloans = spark.read.csv(file,header=True)
DFloans.show()

#In loans abbiamo solo item_id (purtroppo) dunque dobbiamo joinare usando tale attributo E POI procedere
#a una trasformazione.

+--------------------+-------+-------------------+-------------------+-------------------+------------+----------+-----------+
|       patron_id_md5|item_id|    loan_date_begin|      loan_date_end|           due_date|from_library|to_library|end_library|
+--------------------+-------+-------------------+-------------------+-------------------+------------+----------+-----------+
|5772331ac08adcd29...| 837536|2012-06-01 10:56:48|2012-06-01 10:57:32|2012-07-01 10:56:46|          13|        13|         13|
|45d1502e068ccf492...| 353668|2012-01-10 14:55:38|2012-01-10 14:55:58|               null|          16|        27|         27|
|365f660c20c1d8cfe...|1462310|2012-02-08 15:05:34|2012-02-08 15:05:44|               null|          25|         2|          2|
|88222e7e5f8172698...| 741597|2012-02-13 17:00:09|2012-04-10 16:34:04|               null|          15|        18|         18|
|ce746e197d74b1fa7...| 161044|2012-01-12 14:19:16|2012-01-12 14:19:30|               null|          18|        

In [16]:
#RIASSUNTO FILTRAGGI
#TOPOLINO E CO.
#DFmanifestations.show()
DFmanifestationsitems = DFitems.join(DFmanifestations, DFmanifestations.manifestation_id == DFitems.manifestation_id).drop(DFitems.manifestation_id)
#DFmanifestationsitems.show()
DFitemsmagazinefilter = DFmanifestationsitems.filter(~((DFmanifestationsitems.title.contains("Zagor")) | (DFmanifestationsitems.title.contains("Topolino")) | (DFmanifestationsitems.title.contains("Dylan Dog")) | (DFmanifestationsitems.title.contains("Tex")) | (DFmanifestationsitems.title.contains("Paperino")) | (DFmanifestationsitems.title.contains("Disney   Big"))))
#DFitemsmagazinefilter.show()

#SOLO F E B COME CATEGORIE

DFitemscategoryfilter = DFitemsmagazinefilter.filter((DFitemsmagazinefilter.item_media_id == "F") | (DFitemsmagazinefilter.item_media_id == "B"))
#DFitemscategoryfilter.show()

#SOLO LIBRI ITALIANI

DFitemslanguagefilter = DFitemscategoryfilter.filter((DFitemscategoryfilter.edition_language == "ita") | (DFitemscategoryfilter.edition_language == "ITA"))
#DFitemslanguagefilter.show()
#SOLO LIBRI CON MENO DI 50 PRESTITI

DFitemsCFloans = DFitemscategoryfilter.join(DFloans, DFloans.item_id == DFitemscategoryfilter.item_id).drop(DFloans.item_id)
#DFitemsCFloans.show()
DFgroupbytitlesloans = DFitemsCFloans.groupby("title").count().sort("count", ascending=False)
#DFgroupbytitlesloans.show(20, False)
DFlessthan50 = DFgroupbytitlesloans.filter(DFgroupbytitlesloans['count'] < 50)
#DFlessthan50.show(20, False)
DFitemslessthan50filter = DFitemslanguagefilter.join(DFlessthan50, DFlessthan50.title == DFitemslanguagefilter.title, "leftanti")
#DFitemslessthan50filter.filter(DFitemslessthan50filter.title=="Astronomia pratica / Carole Stott").show()

#DFitemslessthan50filter.filter(DFitemslessthan50filter.manifestation_id == "335235").show(20, False)

In [14]:
#PULIZIA AUTORI E TITOLI
def mapping(line):
    author_cleaned = line.author #AUTORI CON UN SOLO NOME (DI SOLITO PSEUDONIMI)
    if line.author != None:
        if len(line.author.split(" - ")) != 1: #AUTORI MULTIPLI
            author_cleaned = ""
            count = 0
            for author in line.author.split(" - "):
                count += 1
                if len(author.split(", ")) == 2:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0]
                else:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[0]
        else:
            if len(line.author.split(",")) == 2: #AUTORI CON NOME E COGNOME SEMPLICI
                author_cleaned = line.author.split(", ")[1] + " " + line.author.split(", ")[0]
            
    else: #L'autore è contenuto nel titolo e non nel campo autore
        author_cleaned = ""
        title_zone_temp = line.title.split(" ; ")[0]
        if len(title_zone_temp.split(" / ")) == 2:
            if len(title_zone_temp.split(" / ")[1].split(" e ")) > 1: #AUTORI SEPARATI DA CONGIUNZIONE E
                title_zone = title_zone_temp.split(" / ")[1].split(" e ")
                count = 0
                for part in title_zone:
                    count += 1
                    if count > 1 and count <= len(title_zone):
                        author_cleaned += " - "
                    words = part.split(" ")
                    count_word = 0 
                    for word in words:
                        count_word += 1
                        string_word = str(word)
                        if len(string_word) > 0:
                            if string_word[0].strip(" [").isupper() == True:
                                if count_word == len(words):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                if author_cleaned.startswith(" -"): #NEL CASO CI FOSSE UNA "e" CHE NON SEPARA DUE AUTORI
                    author_cleaned = author_cleaned.replace(" -", "")
            else:
                title_zone = title_zone_temp.split(" / ")[1].replace("[", "").split(" ")
                if len(title_zone) > 0:
                    count = 0
                    for word in title_zone:
                        string_word = str(word)
                        count += 1
                        if len(string_word) > 0:
                            if string_word[0].isupper() == True:
                                if count == len(title_zone):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                        if len(author_cleaned.split(", ")) > 1: #Autori multipli con virgola
                            templist = author_cleaned.split(", ")
                            author_cleaned = ""
                            for author in templist:
                                string_word = str(author)
                                count += 1
                                if len(string_word) > 0:
                                    if string_word[0].isupper() == True:
                                        if count == len(title_zone):
                                            author_cleaned += string_word
                                        else:
                                            author_cleaned += string_word + " - "
    if author_cleaned.startswith(" "):
        author_cleaned = author_cleaned.replace(" ", "", 1)
    author_cleaned = author_cleaned.replace("*", "")
    author_cleaned = author_cleaned.replace('"', "")
    author_cleaned = author_cleaned.replace(":", "")
    author_cleaned = author_cleaned.replace("!", "")
    author_cleaned = author_cleaned.strip(' []*:') #PER PULIRE LE EVENTUALI PARENTESI QUADRE E SPAZI  
    author_cleaned = re.sub('<[^>]+>', '', author_cleaned)
    author_cleaned = author_cleaned.replace("<", "")
    author_cleaned = re.sub(' +',' ', author_cleaned)
    author_cleaned = author_cleaned.replace("_", " ")
    author_cleaned = author_cleaned.replace(">", "").strip(" ")
    
    #PER ELIMINARE I REFUSI PIù FREQUENTI
    title_cleaned = line.title.split(" / ")[0]
    title_cleaned = title_cleaned.replace("\x89", "").strip(" ")
    title_cleaned = title_cleaned.replace("\x88", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\1", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\2", "").strip(" ")
    title_cleaned = title_cleaned.replace('"', "")
    title_cleaned = title_cleaned.replace('[', "")
    title_cleaned = title_cleaned.replace("\\", "").strip(" ")
    title_cleaned = title_cleaned.replace(']', "")
    title_cleaned = title_cleaned.replace('*', "")
    title_cleaned = title_cleaned.replace('|', "")
    title_cleaned = title_cleaned.replace("!:", "").strip(" ")
    title_cleaned = title_cleaned.replace(" : romanzo", "")
    return Row(item_id=line.item_id, item_media_id=line.item_media_id, manifestation_id=line.manifestation_id,
              title=title_cleaned, author=author_cleaned, publisher=line.publisher)

def specialcasesmapping(line): #Casi strani disseminati nel Database che riparo manualmente
    new_author = line.author
    new_title = line.title
    new_publisher = line.publisher
    if line.manifestation_id == "440293":
        new_author = "Chiara Lossano"
    if line.manifestation_id == "190617":
        new_author = "Italo Calvino"
    if line.manifestation_id == "430234":
        new_publisher = None
        new_author = "Philip Roth"
        new_title = "«Ho sempre voluto che ammiraste il mio digiuno» ovvero, guardando Kafka"
    if line.manifestation_id == "427622":
        new_author = "Shí Héng Chán - Shì Héng Díng"
    return Row(item_id=line.item_id, item_media_id=line.item_media_id, manifestation_id=line.manifestation_id,
              title=new_title, author=new_author, publisher=new_publisher)
        
def maptopair(line):
    return (line.title, list(line))

def reductionbykey(line1, line2):
    if int(line1[3]) < int(line2[3]):
        return Row(title=line1[5], author=line1[0], publisher=line1[4], item_id=line1[1], manifestation_id=line1[3], item_media_id=line1[2])
    else:
        return Row(title=line2[5], author=line2[0], publisher=line2[4], item_id=line2[1], manifestation_id=line2[3], item_media_id=line2[2])

def reductionbykey2(line1, line2):
    if int(line1[3]) < int(line2[3]):
        return line1
    else:
        return line2
    
def maptosingle(line):
    return line[1]

RDDitemslessthan50filter = DFitemslessthan50filter.rdd
RDDmappedtry = RDDitemslessthan50filter.map(mapping)
RDDmappedtrytwo = RDDmappedtry.map(specialcasesmapping)
RDDfilterednoauthornone = RDDmappedtrytwo.filter(lambda line : line.author != '') #LEVIAMO EVENTUALI AUTORI NULLI

RDDreducedtry = RDDfilterednoauthornone.map(maptopair).sortByKey()
RDDreducedtrytwo = RDDreducedtry.reduceByKey(reductionbykey).sortByKey()
RDDreducedtrythree = RDDreducedtrytwo.map(maptosingle)

#for row in RDDreducedtrythree:
#    print(row)
#    #break

DFcleantitles = RDDreducedtrythree.toDF()
DFcleantitles.toPandas().to_csv('titoli_puliti.csv', index=False)

#NOTE PER ME SU LIBRI SOSPETTI
#La vampa d'agosto
#CARNISIO, ROSANNA
#NOME COGNOME - NOME COGNOME 
#Row(item_id='25010', 
#item_media_id='F', 
#home_library_id='16', 
#inventory_date='2009-07-13', 
#manifestation_id='366720', 
#edition_language='ita', 
#edition_date='2008', 
#ISBNISSN='9788804582991', 
#EAN='9788804582991', 
#title='13 / Jay Asher ; traduzione di Stefano Borgotallo', 
#author=None, OPPURE COGNOME, NOME OPPURE COGNOME_COGNOME, NOME <DATA DI NASCITA>
#publisher='Mondadori', 
#loanable_since=None, 
#date_created='2008-11-14 00:00:00')
#Autori nel titolo: TITOLO / AUTORE OPPURE TITOLO / AUTORE ; CURATORI
#Flavio Fogarolo
#Marie-France Floury